In [ ]:
%run forecasting.ipynb

In [2]:
kaggle_test = pd.read_csv("../data/test.csv")
kaggle_test['store_item'] = kaggle_test.store.astype(str) + '-' + kaggle_test.item.astype(str)
kaggle_test = kaggle_test.set_index(pd.to_datetime(kaggle_test.date))
kaggle_test.drop(['id', 'store', 'item', 'date'], axis=1, inplace=True)

In [3]:
testing_models = False
forecast_horizon = 90

if testing_models:
    kaggle_test = kaggle_test[kaggle_test.store_item.isin(['1-1', '1-2', '1-3'])]
    data = data[data.store_item.isin(['1-1', '1-2', '1-3'])]

In [4]:
horizon_end_date = kaggle_test.index[-1]
fcast_begin = kaggle_test.index[0]

In [5]:
kaggle_test["autoreg"] = fit_autoreg(df=data, fcast=fcast_begin, horizion_end=horizon_end_date)

100%|██████████| 500/500 [01:13<00:00,  6.83it/s]


In [6]:
kaggle_test["exp_smooth"] = exp_smooth_predictor(df=data, seas="add", fcast=fcast_begin, horizion_end=horizon_end_date)

100%|██████████| 500/500 [12:24<00:00,  1.49s/it]


In [7]:
kaggle_test["ardl"] = ardl_predictor(data, fcast=fcast_begin, horizion_end=horizon_end_date)

In [8]:
date_list = [kaggle_test.index[-1] + timedelta(days=x+1) for x in range(forecast_horizon)]

kaggle_test["xgb_preds"] = xgb_predictor(data, date_list)

100%|██████████| 500/500 [04:05<00:00,  2.04it/s]


In [ ]:
kaggle_test["prophet"] = prophet_predictor(data, date_list)

In [ ]:
kaggle_test["neural_prophet"] = neural_prophet(data, date_list)

In [11]:
if not testing_models:
    from datetime import date
    today = date.today()
    kaggle_test.to_csv(f"../data/predictions/predictions-final-{today.month}-{today.day}.csv")